# Getting started with Amazon Braket Hybrid Job

This tutorial shows how to run a simple Amazon Braket Hybrid Job on Amazon Braket simulators and quantum devices. 
To get started quickly, we consider simple circuits with only one qubit and one gate in this example notebook.


## Learning Outcomes 
* Run our first Braket Job! 
* Understand how to run scripts or functions
* Monitor job status and view logs 
* Save results from a job
* Use Braket Jobs within a specific AWS session
* Run Braket Jobs on quantum hardware 
* Use Local Braket Jobs to quickly test code [Docker required]


## Algorithm script

To create a Braket Job, we first need a Python script that we want to run. In this example, it's contained in `algorithm_script.py`. The script is printed in the code cell below for convenience. 

As shown, each of our circuits has only one $X$ rotation gate with a random angle. The circuit is repeated five times with different random rotations. We would write the algorithm script as we usually do, except that we do not specify the backend QPUs or simulators explicitly. Instead, it is provided through environment variables which are passed to the algorithm script when creating a Braket Job. 

In [15]:
%%script echo "This block is a copy of the algorithm script."

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result


print("Test job started!")

# Use the device declared in the creation script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list})

print("Test job completed!")

This block is a copy of the algorithm script.


## Creating a Braket Job 

In this notebook, we use <code>AwsQuantumJob.create</code> to create a Braket job. When the Braket job is created, it starts an EC2 instance and spins up a Docker container. The instance type, container and other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks for more information.

The required inputs of <code>AwsQuantumJob</code> are:
- <b>device</b>: The arn of the Braket simulator or QPU we want to use. It will be passed as an environment variable to the algorithm script.
- <b>source_module</b>: The path to a file or a python module that contains your algorithm script. It will be uploaded to the container for running the Braket Job.
- <b>entry point</b>: The path relative to the source_module. It points to the code to be run when the Braket Job starts. 
- <b>wait_until_complete (optional)</b>: If true, the function call will wait until the Braket Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. 

In [16]:
from braket.aws import AwsQuantumJob

In [17]:
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    wait_until_complete=True,
)

TypeError: create() got an unexpected keyword argument 'device'

In this exercise, the algorithm script is in one file, so we set <code>source_module</code> to be <code>algorithm_script.py</code> and <code>entry_point</code> to be <code>algorithm_script</code>. Depending on your application, there are other options for setting the algorithm script. For example, if you wish to only execute a part of <code>algorithm_script.py</code> at the start of a Braket Job, you can package that part to be a <code>starting_function()</code>. Then the input arguments would be

In [18]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

When your algorithm script requires other files such as helper functions, you can put them all in one folder, say the <code>algorithm_folder</code>. The input arguments would then be

In [8]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

## Tracking progress and results

The status of a Braket Job can be checked by calling <code>job.state()</code>. Once completed, the result can be retrieved using <code>job.result()</code>. Logs and metadata are also accessible via `job.logs()` and `job.metadata()`. If you lose the job variable, you can always retrieve it by a unique arn which you can find in the AWS Console.

In [24]:
job.state()

'COMPLETE'

In [25]:
results = job.result()  # will return once  job.state() = "COMPLETE"
print("counts")
print(results["counts"])
print("angles")
print(results["angles"])

counts
[{'0': 73, '1': 27}, {'0': 58, '1': 42}, {'1': 100}, {'1': 100}, {'1': 85, '0': 15}]
angles
[7.45762901541143, -1.4086734055114467, 3.119866923277959, 3.340830403507961, 3.8432630257476528]


In [22]:
%%capture captured
print(job.logs())
print(job.metadata())

In [28]:
job.download_result()  # download job result to local directory

## Running on QPUs

Braket Jobs can run our quantum processing units (QPUS) available through Amazon Braket. Be aware that QPU devices are region specific. 

Running jobs on QPUs has the advantage of priority access to the QPU. Quantum tasks within a single job are grouped together to attempt to run sequentially. This reduces the risk of certain task being delayed or drifting calibrations on the device. 

We can seamlessly swap our SV1 simulator for a QPU by changing the device argument in `AwsQuantumJob.create()`. 

In [26]:
# Make sure the AWS region matched the device availability region.
# This cell could take a long depending on our position in the queue and device availability
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/qpu/rigetti/Aspen-9",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    wait_until_complete=False,
)

TypeError: create() got an unexpected keyword argument 'device'

# AWS Sessions

We can customize where Braket Jobs saves and loads results in Amazon S3 by providing the AWS session information. 

In [27]:
from braket.aws import AwsSession

aws_session = AwsSession(
    default_bucket="amazon-braket-{region}-{account_id}"
)  # Set Amazon S3 bucket

To create a Braket job with this S3 bucket, we pass `aws_session` as an argument to ` AwsQuantumJob.create()`

```
job = AwsQuantumJob.create(
    "arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    aws_session = aws_session # using specific S3 bucket
)
```

## Local Braket Jobs

Braket Jobs can also be run locally before submitting a job to Amazon Braket. This is recommended for quickly debugging and testing code, but does not provide AWS Console information on the job. This feature requires Docker to be installed on your computer. Amazon Braket Notebook Instances have Docker pre-installed, so you can test local jobs in hosted notebooks instantly. To install Docker in your local environment, follow these [instructions](https://docs.docker.com/get-docker/).

Local jobs always run synchronously and display the logs.

In [14]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
)

TypeError: create() got an unexpected keyword argument 'device'

## Summary

In this tutorial, we have created our first Braket Job with a simple batch of five circuits. We learned how to change the Amazon S3 bucket for a job, and how to save results. We learned about running on simulators and QPUs. Lastly, we used local jobs to quickly test code.